<a href="https://colab.research.google.com/github/unizar-flav/DAIProLi/blob/main/DAIProLi_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Determination of Affinity of Interaction between Protein and Ligand

In [ ]:
#@title Modules and functions
#@markdown You need to run this cell only once regardless of the number of datasets to be evaluated.


import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import csv
import zipfile # Necesary to compress the files into zip

from bokeh.io import output_notebook, show, export_png
from bokeh.plotting import figure, output_file, save
from bokeh.models import Legend
from bokeh.palettes import linear_palette, Viridis256
from bokeh.palettes import Category20

from mpl_toolkits.mplot3d import Axes3D

from google.colab import files
from datetime import datetime
import os

# This is not definitive, the final solution will be to upload the module into the Unizar's GitHub
!git clone https://github.com/unizar-flav/DAIProLi.git
from DAIProLi.funcionesGenerales import  procesa, argLeastSquares

##Specific functions


def leeFichero(nombrFich, colValLabel='Wavelength', intercaladas=True, separador=',', column_inter="Name"):
    '''

    '''

    nLineas = 0
    colValues = []  # This will store the original column names
    colnames_inter = [] # This will store the original column names in the case for intercaladas
    matriz = None

    # Open and read the file
    with open(nombrFich, 'r') as fichero:
        for linea in fichero:
            row = np.array(())

            # If we detect the column_inter label, capture the original column names
            if linea.split(separador)[0].strip() == column_inter:
                colnames_inter = [dato.strip() for dato in linea.split(separador) if dato.strip()]
                #print(f"Captured intercalated column names: {colnames_inter}")  # Debugging line

            # If we detect the column value label, capture the original column names
            if colValLabel in linea:
                colValues = [dato.strip() for dato in linea.split(separador) if dato.strip()]
                #print(f"Captured column values: {colValues}")  # Debugging line

                # If intercaladas is True, intercalate the column names as well
                if intercaladas == True:
                    colValues = colnames_inter
                colValues[0] = "Wavelength (nm)"
                continue

            try:
                # Process data lines
                for dato in linea.split(separador):
                    if dato.strip():
                        row = np.append(row, float(dato))

                # Ensure the row contains data before processing
                if len(row) > 0:
                    if intercaladas:
                        # Only intercalate if the row has at least 2 elements
                        if len(row) > 1:
                            row = np.append(row[0], row[1::2])
                        else:
                            row = row  # Leave row as is if it only has 1 element

                    # Initialize the matrix on the first valid data row
                    if nLineas == 0:
                        matriz = np.empty((0, len(row)))

                    # Stack the rows into the matrix
                    matriz = np.vstack((matriz, row))
                    nLineas += 1

            except ValueError:
                pass  # Skip rows that cannot be processed

    # Convert the matrix to a DataFrame
    if matriz is not None:
        df = pd.DataFrame(matriz)

        if intercaladas == True and "Baseline" in colValues: # In this line of code we delete the Baseline column only when intercaladas is true
            col_index = colValues.index("Baseline")
            df.drop(columns=[col_index], inplace=True)
            colValues.pop(col_index)

        # Apply the intercalated column names
        if len(colValues) == df.shape[1]:
            df.columns = colValues
        else:
            print(f"Warning: Mismatch between column names ({len(colValues)}) and data columns ({df.shape[1]}).")
            df.columns = colValues[:df.shape[1]]  # Use available names if mismatch


        return df
    else:
        print("No data was read from the file.")
        return None





def diff_absorbance(df, method="Manual", check_vals=False, min_chosen=0, max_chosen=0):

  # Extract the values of the first column (Wavelength (nm))
  wavelength= df.iloc[:,0]

  # Extract the absorbance measurements (all the columns except for the first one)
  absorbance= df.iloc[:,1:]

  # Extract the columns' names from the third there on (third included)
  volume= df.columns[2:]
  volume = pd.to_numeric(volume)

  # Initialize the lists to record the minimums, maximums and their respective wavelengths
  min_vals = []
  max_vals = []
  min_wave = []
  max_wave = []

  # Iterate over each column (except the first and second, wavelength and baseline respectively)
  for col in df.columns[2:]:
    # Find the minimum, maximum value and their respective wavelengths
      min_val = df[col].min()
      max_val = df[col].max()

      min_wl = df[df[col] == min_val]['Wavelength (nm)'].iloc[0]
      max_wl = df[df[col] == max_val]['Wavelength (nm)'].iloc[0]
      #print(min_wl)
      # Record in the information on their respective lists
      min_vals.append(min_val)
      max_vals.append(max_val)
      min_wave.append(min_wl)
      max_wave.append(max_wl)

  if check_vals == True:
    # Now we create some dataframes to view the inner workings of the function
    inter_min_dict = {"min_wave (nm)":min_wave, "min_abs":min_vals}
    inter_min_df =pd.DataFrame(inter_min_dict, index=volume)
    inter_max_dict = {"max_wave (nm)":max_wave, "max_abs":max_vals}
    inter_max_df =pd.DataFrame(inter_max_dict, index=volume)


    print(f'Results from minima search: \n{inter_min_df}')
    print(f'Results from maxima search: \n{inter_max_df}')



  if method == "Mean":
    # Calculate the mean of the wavelengths belonging to the minimums and maximums
    avg_min_wave = np.mean(min_wave)
    avg_max_wave = np.mean(max_wave)
    # Instead of rounding, find the closest wavelength in the dataset for both avg_min_wave and avg_max_wave
    closest_min_wave = wavelength.iloc[(np.abs(wavelength - avg_min_wave)).idxmin()]
    closest_max_wave = wavelength.iloc[(np.abs(wavelength - avg_max_wave)).idxmin()]
  elif method == "Median":
    closest_min_wave = np.median(min_wave)
    closest_max_wave = np.median(max_wave)
  elif method == "Manual":
    closest_min_wave = wavelength.iloc[(np.abs(wavelength - min_chosen)).idxmin()] # This ensures that a true wavelength is chosen
    closest_max_wave = wavelength.iloc[(np.abs(wavelength - max_chosen)).idxmin()]
  else:
    print("Error")






  # Find the Dataframe's row that corresponds to the "closest" maximum and minimum wavelengths
  min_abs = df[df['Wavelength (nm)'] == closest_min_wave].iloc[:,2:].reset_index(drop=True).T
  max_abs = df[df['Wavelength (nm)'] == closest_max_wave].iloc[:,2:].reset_index(drop=True).T

  # Calculate deltaAbs as the difference between absorbance at the "closest" maximum and minimum wavelengths
  deltaAbs = max_abs - min_abs

  # Create a DataFrame with the results to be the output
  out_df = deltaAbs
  out_df.insert(0, column ='',value=volume)
  out_df.reset_index(drop=True, inplace=True)
  out_df.columns= ['Volume µl', 'ΔAbs']
  # Add two columns to write the "closest" maximum and minimum wavelengths
  out_df["min_wave (nm)"] = closest_min_wave
  out_df["max_wave (nm)"] = closest_max_wave


  # Print the values calculated
  print(f'Minimum:  ({closest_min_wave} nm)\t Maximum:  ({closest_max_wave} nm)')


  return out_df


# Define the function to shift spectra so that they become zero at a specific wavelength

"""
## English
Adjusts all spectra so that their absorbance is zero at a given target wavelength.

Parameters:
wavelengths (dataframe): A dataframe of one column with the wavelength values.
absorbance (dataframe): A dataframe where each column is a spectrum at different volumes.
target_wavelength (float): The wavelength at which all spectra should be adjusted to zero.


shifted_absorbance: The absorbance values adjusted to zero at the target wavelength.

Returns:
shifted_data: A dataframe result of merging wavelengths and shifted_absorbance dataframes


"""
def shift_spectra_to_zero(data, target_wavelength):
    wavelengths=datos.iloc[:,0]
    absorbance=datos.iloc[:,1:]

    # Find the index of the target wavelength in the wavelengths dataframe
    target_idx = (np.abs(wavelengths - target_wavelength)).argmin()

    # Extract the absorbance at the target wavelength for each spectrum
    absorbance_at_target = absorbance.iloc[target_idx, :]

    # Shift each spectrum by subtracting its absorbance at the target wavelength
    shifted_absorbance = absorbance - absorbance_at_target.values
    shifted_data = pd.concat([wavelengths, shifted_absorbance], axis=1)

    return shifted_data



########



In [ ]:
#@title Upload file

#@markdown Select the correspondent choices dependending in your data format,
#@markdown  if in doubt do not change the preset values.

# Upload the file and save in a dictionary
uploaded=files.upload()

# Obtain the uploaded file name from the dictionary
file_name=list(uploaded.keys())[0]
# @markdown **Interspersed Columns**

Interspersed = True #@param {type:"boolean"}

# @markdown **Columns Separator**
Separator = "," #@param [",", ";"]


datos= leeFichero(nombrFich=file_name, intercaladas= Interspersed, separador=Separator)
datos

In [ ]:
#@title diff_absorbance
print("**diff_absorbance:**")

#@markdown Here it is advisable to run the function with
#@markdown the median option first along with the check_vals in
#@markdown order to determine whether a manual input is
#@markdown is necessary or not.







# @markdown **Method for selecting minimum and maximum**
Method = "Median" #@param ["Manual", "Mean", "Median"]

# @markdown **Manual minimum (nm)**
Minimum  = 0 #@param {type:"number"}
# @markdown **Manual Maximum (nm)**
Maximum  = 0 #@param {type:"number"}


# @markdown **Check the minimum and maximum for each spectrum**
Check_vals = True #@param {type:"boolean"}





DeltaAbs= diff_absorbance(datos, method= Method, check_vals=Check_vals, )
DeltaAbs

In [ ]:
#@title Spectra plot
# @markdown In the spaces below you
# @markdown can write the title, the axis and leyend titles


### Title of the plot
Title = "Spectra at Different Volumes of ***"  #@param {type:"string"}

### Title axis x
y_axis = "Absorbance" #@param{type:"string"}

### Title axis y
x_axis = "Wavelength (nm)" #@param{type:"string"}

### Title legend
Leyend = "Volume (µL)" #@param{type:"string"}

### Option to set the spectra at 0 at a specific wavelength
Shift_spectra= True #@param{type:"boolean"}
target_wavelength=600 #@param{type:"raw"}

shifted_absorbance = shift_spectra_to_zero(datos, target_wavelength)

if Shift_spectra == True:
  df = shifted_absorbance
else:
  df = datos
# Check the result
#shifted_absorbance[:]  # Displaying the first few rows of the shifted absorbance



# Output the plot directly in the notebook
output_notebook()

# Create a figure
p = figure(title=Title,
           x_axis_label=x_axis,
           y_axis_label=y_axis,
           width=1200, height=700) # Here you can modify the resolution (size) of the plot

# Define the font size for the title, the axis and labels
p.title.text_font_size = '20pt'
p.xaxis.axis_label_text_font_size = '16pt'
p.yaxis.axis_label_text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'


# Generate a personalized color paletter using Viridis256, with as may colors as columns are in the data
n_lines= len(df.columns[1:])
colors=linear_palette(Viridis256, n_lines)
#colors = Category20[max(3, min(20, n_lines))]  # Category20 supports up to 20 distinct colors


# Iterate over the columns (each one represents a different volume)
for idx, col in enumerate(df.columns[1:]):
    p.line(df.iloc[:, 0], df[col], legend_label=f'{col}', line_width=2, color=colors[idx])

# Personalize the leyend Personalizamos la leyenda
p.legend.title = Leyend
p.legend.location = "top_right"
p.legend.click_policy = "hide"  # Allows to hide the lines by clicking its label in the legend
p.toolbar_location = "below"
# Define the font size for the legend
p.legend.label_text_font_size = '12pt'
p.legend.title_text_font_size = '14pt'


# Save the plot as an object
spectra_plot_2D=p

# Display the plot
show(spectra_plot_2D)


# 3D plot


# Load experimental data
wavelengths = df.iloc[:, 0].values  # Wavelength (nm)

# Para evitar problemas asignamos 0 a "Baseline" y convertirmos el resto a números enteros
# volumes = np.array([0 if col == 'Baseline' else int(col) for col in df.columns[1:]])  # Volumes (µL)

# To avoid problems we asign 0 a "Baseline" and
# For volumes that are floats the line above creates erros
volumes = np.array([0 if col == 'Baseline' else float(col) for col in df.columns[1:]])  # Volumes (µL)


# Absorbance value for each volume (Baseline included)
absorbance = df.iloc[:, 1:].values

# Create a figure
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot each volume spectrum as a 3D line
for i, vol in enumerate(volumes):
    ax.plot(wavelengths, [vol] * len(wavelengths), absorbance[:, i], label=f'{vol} µL')

# Labels and title
ax.set_xlabel(x_axis)
ax.set_ylabel(Leyend)
ax.set_zlabel(y_axis)
ax.set_title(Title)

# Save the plot as an object
spectra_plot_3D=fig

# Display the plot
spectra_plot_3D.show()

In [ ]:
#@title Plot Δ Absorbance vs Volume (µL)


### Plot tittle
Title = "Δ Absorbance vs added Volume ***"  #@param {type:"string"}

### Title axis x
y_axis = "Δ Absorbance" #@param{type:"string"}

### Title axis y
x_axis = "Volume (µL)" #@param{type:"string"}

### Title leyend
Leyend = "Δ Absorbance" #@param{type:"string"}


# Display Bokeh plots in the notebook
output_notebook()

# Create a figure
p = figure(width=1200, height=700, title=Title)

# Add a scatter-plot
scatter = p.scatter(x=DeltaAbs.iloc[:, 0],y=DeltaAbs.iloc[:, 1] ,  legend_label=Leyend, color='blue', size=10)

# Personalize the axis' titles
p.xaxis.axis_label = x_axis
p.yaxis.axis_label = y_axis


# Define the font size for the title, the axis and the labels
p.title.text_font_size = '20pt'
p.xaxis.axis_label_text_font_size = '16pt'
p.yaxis.axis_label_text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

# Add the leyend
p.legend.title = 'Legend'

# Define the font size for the leyend
p.legend.label_text_font_size = '12pt'
p.legend.title_text_font_size = '14pt'
p.legend.location = "bottom_right"
p.toolbar_location = "below"


# Save the plot as an object
D_Absorbance_plot=p

# Display the plot
show(D_Absorbance_plot)



In [ ]:
#@title Model
# Define the fitting function based on the mathematical model
def binding_model (params, v):

    # Extract the parameters from params
    L = params['L']
    V0 = params['V0']
    Lo = params['Lo']
    Ro = params['Ro']
    Kd = params['Kd']
    epsilon = params['epsilon']

    Lt = Lo * v / (V0 + v)
    Rt = Ro * V0 / (V0 + v)
    deltaAbs = epsilon * L * (Lt + Rt + Kd - np.sqrt((Lt + Rt + Kd)**2 - 4 * Lt * Rt)) / 2
    return deltaAbs

In [ ]:
# @title Parameters
# @markdown **Fixed and to be Optimzed Parameters**:
# @markdown Select the fixed parameters and those to be optimized and input the initial estimation:

import numpy as np
# @markdown

# @markdown **- L [cm] (light path length)**
# Use the dropdown to chose whether the parameters are fixed or not
L_fixed = "Yes"  # @param ["Yes", "No"]
L = 1  # @param {type:"number"}

# @markdown

# @markdown **- V0 [µL] (initial solution volume)**
V0_fixed = "Yes"  # @param ["Yes", "No"]
V0 = 1000  # @param {type:"number"}

# @markdown

# @markdown **- L0 [µM] (initial solution concentration of Ligand)**
Lo_fixed = "Yes"  # @param ["Yes", "No"]
Lo = 55  # @param {type:"number"}

# @markdown

# @markdown **- R0 [µM] (initial solution concentration of Receptor)**
Ro_fixed = "No"  # @param ["Yes", "No"]
Ro = 2  # @param {type:"number"}

# @markdown

# @markdown **- Kd**
Kd_fixed = "No"  # @param ["Yes", "No"]
Kd = 100  # @param {type:"number"}

# @markdown

# @markdown **- epsilon**
epsilon_fixed = "No"  # @param ["Yes", "No"]
epsilon = 0.1  # @param {type:"number"}

# Initialize list for the fixed and variable parameters (these last ones are to be optimized later)
fixed_params = {}
variable_params = {}

if L_fixed == "Yes":
    fixed_params['L'] = L
else:
    variable_params['L'] = L

if V0_fixed == "Yes":
    fixed_params['V0'] = V0
else:
    variable_params['V0'] = V0

if Lo_fixed == "Yes":
    fixed_params['Lo'] = Lo
else:
    variable_params['Lo'] = Lo

if Ro_fixed == "Yes":
    fixed_params['Ro'] = Ro
else:
    variable_params['Ro'] = Ro

if Kd_fixed == "Yes":
    fixed_params['Kd'] = Kd
else:
    variable_params['Kd'] = Kd

if epsilon_fixed == "Yes":
    fixed_params['epsilon'] = epsilon
else:
    variable_params['epsilon'] = epsilon

# Output of the fixed and variable parameters
print("Fixed Parameters:")
for key, value in fixed_params.items():
    print(f"{key} = {value}")

print("\nVariable Parameters:")
for key, value in variable_params.items():
    print(f"{key} = {value}")


In [ ]:
#@title Procesa
# Prepare the list of the parameters names to be opimized
nombrParVar = list(variable_params.keys())

# Independent values (v) and dependent values (Absorbance)
fKwargs = dict(v = DeltaAbs.iloc[:, 0].values) # Data of ligand solution volume added
deltaAbs_exp = DeltaAbs.iloc[:, 1].values # Absorbance data

initial_params= {**fixed_params, **variable_params}
cotaInf = [0 for param in nombrParVar]
cotaSup= [2 if param =='Ro' else np.inf for param in nombrParVar]

sol=procesa(argLeastSquares = argLeastSquares,
                     dictParEstim = initial_params,
                     nombrParVar = nombrParVar,
                     f = binding_model,
                     fKwargs = fKwargs,
                     Y = deltaAbs_exp,
                     bounds =[cotaInf, cotaSup])


In [ ]:
# @title Plot Δ Absorbance vs Volume (µL) with model fitting



### Plot title
Titulo = "Δ Absorbance vs added Volume ***"  #@param {type:"string"}

### Title x-axis
y_axis = "Δ Absorbance" #@param{type:"string"}

### Title y-axis
x_axis = "Volume (µL)" #@param{type:"string"}

### Title leyend
Leyenda = "Δ Absorbance" #@param{type:"string"}


# Display Bokeh plots in the notebook
output_notebook()

# Simulated data using the model with the adjusted parameters
v = DeltaAbs.iloc[:, 0].values
fitted_deltaAbs = binding_model(sol['parAjustados'], v) # Volume's data
deltaAbs_exp= DeltaAbs.iloc[:,1].values

# Create a dataframe to save the values of fitted_DeltaAbs
DeltaAbs_model=DeltaAbs.copy() # Create a copy of DeltaAbs
DeltaAbs_model['ΔAbs']= fitted_deltaAbs # Asign the values of ΔAbs from the model




# Create a figure
p = figure(width=1200, height=700, title=Titulo)

# Add a scatter-plot (experimental data)
scatter = p.scatter(x=v,y=deltaAbs_exp ,  legend_label=Leyenda, color='blue', size=10)

# Add a line with the model fitting
p.line(x=v, y=fitted_deltaAbs, legend_label='Ajuste del modelo', color='red', line_width=3)

# Define the titles of the axis
p.xaxis.axis_label = x_axis
p.yaxis.axis_label = y_axis

# Define the font size for the title, axis and labels
p.title.text_font_size = '20pt'
p.xaxis.axis_label_text_font_size = '16pt'
p.yaxis.axis_label_text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

# Add the leyend
p.legend.title = 'Legend'

# Define the font size for the leyend
p.legend.label_text_font_size = '12pt'
p.legend.title_text_font_size = '14pt'
p.legend.location = "bottom_right"
p.toolbar_location = "below"

# Save the plot as an object
D_Absorbance_plot_fitted=p

# Display the plot
show(D_Absorbance_plot_fitted)


In [ ]:
#@title Download data
#@markdown Write the name for the zip file that contains the data inputted and produced.

# Now let's record all the data that it has been inputted and generated to export it.

# Initial data
datos.to_csv('Experimental_data.csv', index=False)

# Experimental data of Δ Absorbance vs Volume
DeltaAbs.to_csv('DeltaAbs_expertimental.csv', index=False)

# Fitting model data of Δ Absorbance vs Volume
DeltaAbs_model.to_csv('DeltaAbs_fitted.csv', index=False)



# Now let's save the plots

# Plot 2D spectra
output_file("spectra_plot_2D.html")
save(spectra_plot_2D, title="spectra_plot_2D",)

# Plot 3D spectra
spectra_plot_3D.savefig('spectra_plot_3D.png', format='png', dpi=400) # Ajustar dpi para ajustar la resolución

# Plot Δ Absorbance vs Volume (µL) with model fitting
output_file("D_Absorbance_plot_fitted.html")
save(D_Absorbance_plot_fitted, title="D_Absorbance_plot_fitted",)


# Then we edit the data generated by the fitting to save and export them

# To save the results we create two dictionariesPara guardar los resultados vamos a crear dos diccionarios a partir de la solución de procesa
# y de los diccionarios de los parametros iniciales (fijos y variables)

# To save the results we create a dictinary that includes the initial parameters and the result from
# the procesa function (sol)

# List of keys to extract from the sol dictionary
keys= ['parAjustados', 'sdPar', 'R2', 'detalles']

partial= {key:sol[key] for key in keys}

Initial_params={'initialPar':initial_params}

results={**Initial_params, **partial}

print(results)

# Create CSV
with open('Fitting_result.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    # Add blank rows
    writer.writerow([''] * 7)
    writer.writerow([''] * 7)

    # Headers
    writer.writerow(['initialPar', '', '', 'parAjustados', '', '', 'sdPar'])

    # Row 1: L, V0, Lo (verify whether they are optimized or not, and if the do not have sdPar)
    writer.writerow([
        'L', results['initialPar']['L'], '', 'Ro', results['parAjustados']['Ro'], '',
        'Ro_std', results['sdPar'].get('Ro_std', '') if 'Ro' in variable_params else ''  # Display Ro_std only if it is a variable parameter
    ])
    writer.writerow([
        'V0', results['initialPar']['V0'], '', 'Kd', results['parAjustados']['Kd'], '',
        'Kd_std', results['sdPar'].get('Kd_std', '') if 'Kd' in variable_params else ''  # Display Kd_std only if it is a variable parameter
    ])
    writer.writerow([
        'Lo', results['initialPar']['Lo'], '', 'epsilon', results['parAjustados']['epsilon'], '',
        'epsilon_std', results['sdPar'].get('epsilon_std', '') if 'epsilon' in variable_params else ''  # Display epsilon_std only if it is a variable parameter
    ])
    writer.writerow([
        'Ro', results['initialPar']['Ro'], '', 'L', results['parAjustados']['L'], '',
        'L_std', results['sdPar'].get('L_std', '') if 'L' in variable_params else ''  # Display L_std only if it is a variable parameter
    ])
    writer.writerow([
        'Kd', results['initialPar']['Kd'], '', 'V0', results['parAjustados']['V0'], '',
        'V0_std', results['sdPar'].get('V0_std', '') if 'V0' in variable_params else ''  # Display V0_std only if it is a variable parameter
    ])
    writer.writerow([
        'epsilon', results['initialPar']['epsilon'], '', 'Lo', results['parAjustados']['Lo'], '',
        'Lo_std', results['sdPar'].get('Lo_std', '') if 'Lo' in variable_params else ''  # Display Lo_std only if it is a variable parameter
    ])

    # Add blank rows
    writer.writerow([''] * 7)
    writer.writerow([''] * 7)

    # R2 Section
    writer.writerow(['R2'])
    writer.writerow(['R2', results['R2']])
    writer.writerow([''] * 7)
    writer.writerow([''] * 7)

    # Detalles Section
    writer.writerow(['Detalles'])

    # x values (convert floats to strings to concatenate with 'x')
    writer.writerow(['x'] + [str(x) for x in results['detalles']['x']])

    # cost
    writer.writerow(['cost', results['detalles']['cost']])

    # Add blank rows
    writer.writerow([''] * 7)

    # fun values (split into multiple rows)
    fun_values = results['detalles']['fun']
    writer.writerow(['fun'] + [str(f) for f in fun_values[:6]])
    writer.writerow(['fun'] + [str(f) for f in fun_values[6:]])

    # Add blank rows
    writer.writerow([''] * 7)

    # jac values (split into multiple rows)
    for row in results['detalles']['jac']:
        writer.writerow(['jac'] + [str(v) for v in row])

    # Add blank rows
    writer.writerow([''] * 7)

    # grad values (convert floats to strings)
    writer.writerow(['grad'] + [str(g) for g in results['detalles']['grad']])

    # optimality
    writer.writerow(['optimality', results['detalles']['optimality']])

    # active_mask (convert integers to strings)
    writer.writerow(['active_mask'] + [str(a) for a in results['detalles']['active_mask']])

    # nfev, njev, status, message, success
    writer.writerow(['nfev', results['detalles']['nfev']])
    writer.writerow(['njev', results['detalles']['njev']])
    writer.writerow(['status', results['detalles']['status']])
    writer.writerow(['message', results['detalles']['message']])
    writer.writerow(['success', results['detalles']['success']])



# Then we proceed to save all the files and compressed them into a zip file

# Take the current date and hour ()
current_time = datetime.now().strftime("%d%m%Y%H%M%S")

# Define the prefix and create the complete name of the zip file
name ="spectra_" #@param {type: "string"}
zip_filename = f"{name}{current_time}.zip"

# Create a zip file with the name written
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # Add CSV files
    zipf.write('Experimental_data.csv')
    zipf.write('DeltaAbs_expertimental.csv')
    zipf.write('DeltaAbs_fitted.csv')
    zipf.write ('Fitting_result.csv')
    # Add HTML files (Bokeh plots)
    zipf.write('spectra_plot_2D.html')
    zipf.write('D_Absorbance_plot_fitted.html')

    # Add PNG file (3D spectra plot)
    zipf.write('spectra_plot_3D.png')


# Download the zipped file
files.download(zip_filename)
